Install Dependencies

In [ ]:
!pip install langchain

In [ ]:
!pip install openai

Connect to LLM

In [ ]:
# from langchain.chat_models import ChatOpenAI

# llm = ChatOpenAI(openai_api_base = "http://localhost:1234/v1", openai_api_key = "lm_studio", model = "qwen3-0.6b",temperature=0.8,model_kwargs={"seed": 42}) #max_tokens=100

In [65]:
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(openai_api_base = "https://openrouter.ai/api/v1", openai_api_key = "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", model = "deepseek/deepseek-chat-v3.1:free",temperature=0.9) #max_tokens=100

In [ ]:
# from langchain_community.llms import Ollama
# llm = Ollama(model="llama3.2:1b")

Remove thinking Section

In [45]:
import re

def remove_think_sections(text):
    return re.sub(r"<think>.*?</think>", "", text)

In [3]:
text="What would be a good company name for a company that makes colorful socks?"

Basic Run 

In [ ]:
output = llm.predict(text)
output = remove_think_sections(output)
print(output)

Prompt Template

In [ ]:
from langchain.prompts import PromptTemplate

template = "I want to open a restaurant for {cuisine} food. Suggest a fency name for this."

prompt = PromptTemplate(
    input_variables=['cuisine'],
    template=template
)

formatted_prompt = prompt.format(cuisine="Chinese")
print(formatted_prompt)

Few Shot Prompts

In [ ]:
from langchain.prompts import FewShotPromptTemplate


examples = [
    {"word": "cat", "definition": "A small domesticated carnivorous mammal."},
    {"word": "dog", "definition": "A domesticated carnivorous mammal that typically has a long snout."},
]

example_prompt = PromptTemplate(
    input_variables=["word", "definition"],
    template="Word: {word}\nDefinition: {definition}\n"
)

few_shot_prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix="Keep the answer very short and consise and Define the following words:",
    suffix="Word: {input}\nDefinition:",
    input_variables=["input"],
)

output = (few_shot_prompt.format(input="elephant"))
print (output)

In [ ]:
output = llm.predict(output)
output = remove_think_sections(output)
print(output)

Simple Chain

In [ ]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI


prompt = PromptTemplate(
    input_variables=["topic"],
    template="Give me a tweet idea about {topic}"
)

chain = LLMChain(llm=llm, prompt=prompt)

response = chain.run("AI in education")
response = remove_think_sections(response)
print(response)

Sequential Chain

In [ ]:
from langchain.chains import SequentialChain

# Prompt 1: Generate title
title_prompt = PromptTemplate(
    input_variables=["topic"],
    template="Write a compelling blog post title about {topic}. only give one one title as the output nothing more than that"
)
title_chain = LLMChain(llm=llm, prompt=title_prompt, output_key="title")

# Prompt 2: Use title to generate intro
intro_prompt = PromptTemplate(
    input_variables=["title"],
    template="Write a short blog introduction for the post titled: {title}. only give the introduction as the output nothing more than that. have bullet points and emojis in the intro"
)
intro_chain = LLMChain(llm=llm, prompt=intro_prompt, output_key="intro")

# Chain them together
overall_chain = SequentialChain(
    chains=[title_chain, intro_chain],
    input_variables=["topic"],
    output_variables=["title", "intro"]
)

result = overall_chain.invoke("LangChain for AI-powered apps")
# result = remove_think_sections(result)
print(result)

In [ ]:
print(result["intro"])

Memory

In [ ]:
from langchain.chains import ConversationChain
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory


llm = ChatOpenAI(openai_api_base = "http://localhost:1234/v1", openai_api_key = "lm_studio", model = "qwen3-0.6b",temperature=0.9) #max_tokens=100

memory = ConversationBufferMemory()

conversation = ConversationChain(
    llm=llm,
    memory=memory,
    verbose=True,  # shows the input/output/logs
)

print (conversation.predict(input="Hi, my name is Deepan."))


print (conversation.predict(input="What’s my name?"))



Custom Parser

In [ ]:
from langchain.chains import ConversationChain
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.schema import BaseOutputParser


llm = ChatOpenAI(openai_api_base = "http://localhost:1234/v1", openai_api_key = "lm_studio", model = "qwen3-0.6b",temperature=0.9) #max_tokens=100

memory = ConversationBufferMemory()

class RemoveThinkParser(BaseOutputParser):
    def parse(self, text: str) -> str:
        # remove <think>...</think> blocks
        return re.sub(r"<think>.*?</think>", "", text, flags=re.DOTALL).strip()
    
conversation = ConversationChain(
    llm=llm,
    memory=memory,
    verbose=True,
    output_parser=RemoveThinkParser()  
)

print (conversation.predict(input="Hi, my name is Deepan."))


print (conversation.predict(input="What’s my name?"))



Custom Prompt template

In [ ]:
from langchain.chains import ConversationChain
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.schema import BaseOutputParser


memory = ConversationBufferMemory()

class RemoveThinkParser(BaseOutputParser):
    def parse(self, text: str) -> str:
        # remove <think>...</think> blocks
        return re.sub(r"<think>.*?</think>", "", text, flags=re.DOTALL).strip()
    
custom_prompt = PromptTemplate(
    input_variables=["history", "input"],
    template="""
You are a helpful and concise assistant.

{history}
Human: {input}
AI:"""
)

conversation = ConversationChain(
    llm=llm,
    memory=memory,
    verbose=True,
    prompt=custom_prompt,
    output_parser=RemoveThinkParser() 
)


print (conversation.predict(input="Hi, my name is Deepan."))


print (conversation.predict(input="What’s my name?"))



Conversation Summary Memory

In [ ]:
import re
from langchain.chains import ConversationChain
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.schema import BaseOutputParser

from langchain.memory import ConversationSummaryMemory

# llm = ChatOpenAI(openai_api_base = "http://localhost:1234/v1", openai_api_key = "lm_studio", model = "llama-3.2-1b-instruct",temperature=0.9) #max_tokens=100


summary_memory = ConversationSummaryMemory(llm=llm)


memory = ConversationBufferMemory()

class RemoveThinkParser(BaseOutputParser):
    def parse(self, text: str) -> str:
        # remove <think>...</think> blocks
        return re.sub(r"<think>.*?</think>", "", text, flags=re.DOTALL).strip()
    
custom_prompt = PromptTemplate(
    input_variables=["history", "input"],
    template="""
You are a helpful and concise assistant.

{history}
Human: {input}
AI:"""
)

conversation = ConversationChain(
    llm=llm,
    memory=summary_memory,
    verbose=False,
    prompt=custom_prompt,
    output_parser=RemoveThinkParser() 
)


print (conversation.predict(input="Hi, my name is Deepan."))

print (conversation.predict(input="i want to learn about AI and Machine learning."))

print (conversation.predict(input="how can you help me?"))

print (conversation.predict(input="What’s my name?"))


Memory with Chains

In [ ]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

prompt = PromptTemplate(
    input_variables=["input"],
    template="The user said: {input}"
)

chain_with_memory = LLMChain(
    llm=llm,
    prompt=prompt,
    memory=ConversationBufferMemory()
)

chain_with_memory.predict(input="Hello there!")
chain_with_memory.predict(input="What did I just say?")

Agents

In [ ]:
!pip install google-search-results

In [ ]:
!pip install numexpr

In [ ]:
import os
from langchain.chat_models import ChatOpenAI
from langchain.agents import AgentType, initialize_agent, load_tools

# set serpapi key
os.environ["SERPAPI_API_KEY"] = "42a63592c032fc7a97f972c068c18c2055f7409b1886d53270e4e77d089fd71f"

# # local model
# llm = ChatOpenAI(
#     openai_api_base="http://localhost:1234/v1",
#     openai_api_key="lm_studio",
#     model="llama-3.2-1b-instruct",
#     temperature=0.9
# )

# tools
tools = load_tools(["serpapi", "llm-math"], llm=llm) 

# agent
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION, 
    verbose=True,
    handle_parsing_errors=True
)

# test
print(agent.run("what the bitcoin price today and convert it into INR?"))

In [ ]:
!pip install wikipedia

In [ ]:
import os
from langchain.chat_models import ChatOpenAI
from langchain.agents import AgentType, initialize_agent, load_tools


# llm = ChatOpenAI(
#     openai_api_base="http://localhost:1234/v1",
#     openai_api_key="lm_studio",
#     model="llama-3.2-1b-instruct",
#     temperature=0.9
# )

# tools (wikipedia + math)
tools = load_tools(["wikipedia", "llm-math"], llm=llm)


agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,  
    verbose=True,
    handle_parsing_errors=True  
)


print(agent.run("What is this year raised to the 0.43 power?"
                ))

#In what year was the film Departed with Leonardo DiCaprio released?
# "What is this year raised to the 0.43 power?"


Callback

In [ ]:
from langchain.callbacks.base import BaseCallbackHandler

class MyCustomHandler(BaseCallbackHandler):
    def on_llm_start(self, serialized, prompts,**kwargs):
        print(f"LLM is starting with prompt: {prompts}")

    def on_llm_end(self, response, **kwargs):
        print(f"LLM finished. Response: {response}")

    def on_llm_error(self, error, **kwargs):
        print(f"LLM errored: {error}")

In [77]:
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(
    openai_api_base="http://localhost:1234/v1",
    openai_api_key="lm_studio",
    model="llama-3.2-1b-instruct",
    callbacks=[MyCustomHandler()]
)

print(llm.predict("Hello!"))

LLM is starting with prompt: ['Human: Hello!']
LLM finished. Response: generations=[[ChatGeneration(text='How can I assist you today?', generation_info={'finish_reason': 'stop', 'logprobs': None}, message=AIMessage(content='How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 7, 'prompt_tokens': 12, 'total_tokens': 19, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'llama-3.2-1b-instruct', 'system_fingerprint': 'llama-3.2-1b-instruct', 'finish_reason': 'stop', 'logprobs': None}, id='run-e8ec6aaa-df57-454e-bbd6-a151d58084f4-0'))]] llm_output={'token_usage': {'completion_tokens': 7, 'prompt_tokens': 12, 'total_tokens': 19, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'llama-3.2-1b-instruct', 'system_fingerprint': 'llama-3.2-1b-instruct'} run=None
How can I assist you today?
